**чатбот на рекуррентных нейросетях (Keras+TensorFlow)**

Евгений Борисов <borisov.e@solarl.ru>


----

## Библиотеки

In [1]:
import numpy as np
import re

import pandas as pd
pd.options.display.max_colwidth = 200  


In [2]:
def pp(d): return "{:,.0f}".format(d).replace(",", " ")
def ppr(d): print('записей:', pp(len(d)) )  

## Учебные данные

In [3]:
# with open('../data/text/rus-eng/rus.txt','rt',encoding='utf8') as f: 
#     pair = pd.DataFrame([ p.split('\t') for p in f.read().split('\n') if p.strip() ])

In [4]:
with open('../data/text/pairs.txt','rt',encoding='utf8') as f: 
    pair = pd.DataFrame([ p.split('%%') for p in f.read().split('\n') if p.strip() ])

In [5]:
pair.columns=['Q','A']
pair['Q'] = pair['Q'].str.strip()
pair['A'] = pair['A'].str.strip()

In [6]:
ppr(pair)
pair.sample(9)

записей: 1 509


,Q,A
1073,Что с вами?,Не смотрите на меня.
1320,чувство свободы должно быть связано с сознанием ответственности.,"Извини, но ты бываешь иногда немножко нудным."
1124,"Ага, значит, вы все-таки получили удовольствие.","Разумеется. Я устроена, как все."
92,"Мне показалось, что он немножко охладел к тебе. Он, конечно, обиделся.","милая, тебе не следовало из-за такого пустяка портить с ним отношения."
597,"Очевидно, вы плохо знаете своих знакомых. Прощайте.","Прощайте. Вы еще тут? А я боялся, что вы ушли."
1369,А твой адвокат поступает по-мужски?,"Отнесись к этому проще. Это ведь ничто, просто легкое увлечение."
1258,"В самый раз. Интересно, кто еще будет среди ""звезд""?..","Пей, остынет."
679,Потому что это было нужно.,"Да, я вел себя ужасно. Скажи, почему ты ко мне подошла? Только честно."
614,Что тогда?,Ничего.


## Чистим тексты

In [7]:
pair['Q_clean'] = pair['Q'].str.lower()
pair['Q_clean'] = pair['Q_clean'].str.replace(r'([,.?!])', r' \1 ')

# pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\([,.?!]\)', ' \1 ', s))
#pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
#pair['Q_clean'] = pair['Q_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 

pair['A_clean'] = pair['A'].str.lower()
pair['A_clean'] = pair['A_clean'].str.replace(r'([,.?!])', r' \1 ')

# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'(\W)', ' \1 ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\W', ' ', s))
# pair['A_clean'] = pair['A_clean'].apply(lambda s: re.sub( r'\b\d+\b', ' digit ', s)) 

In [46]:
w2v_size = 128

# PAD = np.zeros(w2v_size)
# GO = np.ones(w2v_size)
# EOS = np.ones(w2v_size)*-1



(128,)

In [8]:
# добавляем "служебные" слова - начало и конец последовательности
pair['Q_clean'] = pair['Q_clean'].str.split() + ['<START>']
pair['A_clean'] = ['<GO>'] + pair['A_clean'].str.split() + ['<EOS>']

In [9]:
pair[['Q_clean','A_clean']].sample(9)

,Q_clean,A_clean
155,"[а, твой, кем, работает, ?, <START>]","[<GO>, тоже, шофером, ., <EOS>]"
321,"[а, теперь, примерю, платье, ., ну, как, ?, <START>]","[<GO>, чудесно, ., <EOS>]"
199,"[ну, и, живи, себе, ., только, почему, с, моим, ?, или, тебе, муж, обязательно, с, готовым, инфарктом, нужен, ?, <START>]","[<GO>, лучше, с, инфарктом, ,, чем, ни, с, кем, ., <EOS>]"
1291,"[что, хорошо, ?, <START>]","[<GO>, то, ,, что, ты, сказал, ., ты, всегда, говоришь, правильно, и, разумно, ., я, недаром, тобой, горжусь, ., <EOS>]"
1305,"[можешь, передать, на, словах, что, нужно, ., <START>]","[<GO>, хорошо, ,, только, постарайся, все, запомнить, ., <EOS>]"
1219,"[будешь, шить, для, себя, ., <START>]","[<GO>, для, нас, ., <EOS>]"
1323,"[разговор, будет, долгий, ?, <START>]","[<GO>, не, знаю, ., думаю, ,, не, очень, ., <EOS>]"
876,"[теперь, тебе, нужно, есть, больше, фруктов, ., <START>]","[<GO>, почему, ?, <EOS>]"
312,"[тебе, не, нравится, новая, тахта, ?, <START>]","[<GO>, нет, ,, почему, же, ., отличная, тахта, <EOS>]"


---

In [10]:
# считаем количество слов
pair['lenQ'] = pair['Q_clean'].str.len()
pair['lenA'] = pair['A_clean'].str.len()
pair.describe()

,lenQ,lenA
count,1509.000000,1509.000000
mean,9.021869,9.194168
std,4.191715,4.376973
min,3.000000,3.000000
25%,6.000000,6.000000
50%,8.000000,8.000000
75%,11.000000,12.000000
max,27.000000,27.000000


In [11]:
# определяем максимальную длинну последовательности
pair['lenQ'].quantile(0.95),  pair['lenA'].quantile(0.95)

(18.0, 18.0)

In [12]:
# # выбираем последовательности средней длинны
# # sent_len_min, sent_len_max = 7,10
# sent_len_min, sent_len_max = 5,12

# ppr(pair)
# pair = pair[
#     pair['lenQ'].between(sent_len_min,sent_len_max) 
#     & pair['lenA'].between(sent_len_min,sent_len_max) 
#   ]
# ppr(pair)

sent_len_a_max = pair['lenA'].max()
sent_len_q_max = pair['lenQ'].max()

In [13]:
# выстраиваем входные последовательности в обратном порядке
# и выравниваем длинну последовательностей,
# дополняем короткие словом "служебным" словом,
pad = ['<PAD>']*sent_len_q_max
# pair['Q_clean'] = pair['Q_clean'].apply( lambda t: pad[len(t):] + list(reversed(t)) )
pair['Q_clean'] = pair['Q_clean'].apply( lambda t: t + pad[len(t):] )

pad = ['<PAD>']*sent_len_a_max
pair['A_clean'] = pair['A_clean'].apply( lambda t: t + pad[len(t):] )

In [14]:
pair[['Q_clean','A_clean']].sample(9)

,Q_clean,A_clean
1284,"[ты, замолчишь, когда-нибудь, ?, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, но, ты, же, сам, просил, меня, о, чем-то, говорить, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
1280,"[а, пока, ты, не, дашь, мне, немного, на, билет, ?, понимаешь, ,, получка, послезавтра…, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, этого, хватит, ?, <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
1363,"[с, помощью, телекоммуникации, ., ну, как, ,, не, скучно, ?, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, скучно, ., обещал, показать, кино, ,, а, сам, прокручиваешь, чьи-то, незнакомые, голоса, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
1374,"[после, выборов, я, захлопну, перед, ними, дверь, ., это, быдло, будет, мне, только, мешать, ., <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, сначала, надо, выиграть, выборы, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
1361,"[и, все, же, не, стоит, торопиться, ., он, нам, может, еще, пригодиться, ., <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, зачем, ?, обойдемся, теперь, и, без, него, ., ты, уже, без, пяти, минут, депутат, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
1219,"[будешь, шить, для, себя, ., <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, для, нас, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
587,"[а, почему, нет, ?, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, но, это, было, бы, просто, смешно, !, <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
611,"[но, мы, пойдем, ?, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, пойдем, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"
325,"[можно, тебя, обнять, ?, <START>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]","[<GO>, не, надо, ,, помнешь, платье, ., <EOS>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>, <PAD>]"


## Кодируем тексты

In [15]:
# data = pair['Q_clean'].values.tolist() + pair['A_clean'].values.tolist()

In [16]:
%%time

from gensim.models.word2vec import Word2Vec

# w2v_size = 128

w2v_q = Word2Vec( pair['Q_clean'].values.tolist(), min_count=1, size=w2v_size, window=4, workers=4)
w2v_a = Word2Vec( pair['A_clean'].values.tolist(), min_count=1, size=w2v_size, window=4, workers=4)

CPU times: user 455 ms, sys: 19.6 ms, total: 474 ms
Wall time: 404 ms


In [17]:
w2v_q_vocab = sorted([w for w in w2v_q.wv.vocab])
ppr(w2v_q_vocab)

w2v_a_vocab = sorted([w for w in w2v_a.wv.vocab])
ppr(w2v_a_vocab)

записей: 2 749
записей: 2 452


In [18]:
ii = np.random.permutation(len(w2v_q_vocab))[:10]
for i in ii:
    w = w2v_q_vocab[i]
    ww = [ v[0] for v in w2v_q.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

погода : ['от', 'я', 'вам', 'же', 'за']
мужчин : ['знаете', 'сразу', 'где', 'ваш', 'во']
характера : ['прощайте', 'сумме', 'моему', 'нелепо', 'твою']
можно : ['чтобы', 'по', 'разве', 'будем', 'зачем']
показывал : ['жить', 'об', 'всего', 'ему', 'вам']
подойти : ['там', 'много', 'чему', 'стихи', 'извини']
цвет : ['настроение', 'нервничаешь', 'мармелад', 'документах', 'работаете']
какая-то : ['по', 'чтобы', 'ни', 'разве', 'можно']
ненавидела : ['коньячку', 'ответа', 'милуетесь', 'погоде…', 'ушел']
жива : ['кажется', 'возникают', 'последнее', 'поневоле', 'разнообразия']


/usr/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [19]:
ii = np.random.permutation(len(w2v_a_vocab))[:10]
for i in ii:
    w = w2v_a_vocab[i]
    ww = [ v[0] for v in w2v_a.wv.most_similar(w, topn=5) ]
    print( w,':',ww )

трудно : ['врач', 'нельзя', 'случае', 'тому', 'говорит']
радостей : ['жизнь', 'нам', 'этот', 'ни', 'там']
затюкали : ['давайте', 'стараться', 'тобой', 'смотреть', 'ни']
телекоммуникации : ['да', 'могу', 'не', '<GO>', 'тоже']
минутку : ['буду', 'говорю', 'замуж', 'угодно', 'наверное']
бога : ['немного', 'хочешь', 'должен', 'ну', 'до']
удобная : ['нет', '<GO>', 'не', 'да', 'могу']
мужчинами : ['никогда', 'нужно', 'же', 'хорошо', 'в']
свободен : ['после', 'ваши', 'любовь', 'первой', 'смотрю']
ненавижу : ['мой', 'закрывается', 'твоего', 'проснуться', 'тогда']


---

In [20]:
# c = w2v_a.wv['<GO>']
# w2v_a.wv.similar_by_vector(c)[0][0] 
# c.min(), c.max()

---

In [21]:
pair['Q_code'] = pair['Q_clean'].apply(lambda t: [ w2v_q.wv.get_vector(w) for w in t ] )
pair['A_code'] = pair['A_clean'].apply(lambda t: [ w2v_a.wv.get_vector(w) for w in t ] )

In [22]:
pair[['Q_code','A_code']].sample(3)

,Q_code,A_code
421,"[[0.056845702, 0.021039685, 0.01196952, -0.013683005, -0.022696037, -0.016484795, -0.023952417, 0.01550289, 0.027040591, 0.0026980876, 0.0026110776, 0.0026532586, 0.007910195, 0.037658207, 0.00158...","[[0.80658114, -0.030691996, 0.54139835, -0.7007458, 0.07014995, -0.305851, -0.3034, -0.04372339, 0.17982915, 0.3023216, 0.32000008, 0.60582095, -0.046074145, 0.4768631, 0.08424453, -0.28807694, 0...."
1303,"[[0.7255033, 0.18626045, 0.26127172, -0.20711544, -0.23588714, -0.092579775, -0.32615766, 0.16111612, 0.2855274, 0.06681968, 0.070837714, -0.054886654, 0.106872775, 0.400361, 0.06850821, -0.146008...","[[0.80658114, -0.030691996, 0.54139835, -0.7007458, 0.07014995, -0.305851, -0.3034, -0.04372339, 0.17982915, 0.3023216, 0.32000008, 0.60582095, -0.046074145, 0.4768631, 0.08424453, -0.28807694, 0...."
233,"[[-0.0020107445, 0.007980947, -0.010506567, 0.00031411846, -0.015274703, -0.0134443175, 0.006128367, 0.0041307593, 0.0074409074, -0.00247137, 0.0018576186, 0.011120703, -0.005931086, 0.0015459345,...","[[0.80658114, -0.030691996, 0.54139835, -0.7007458, 0.07014995, -0.305851, -0.3034, -0.04372339, 0.17982915, 0.3023216, 0.32000008, 0.60582095, -0.046074145, 0.4768631, 0.08424453, -0.28807694, 0...."



-----

In [23]:
ppr(pair)

записей: 1 509


In [24]:
# pair = pair.sample(1000)

# pair = pair.sample(283800)
# pair = pair.sample(600)

In [25]:
encoder_input_data = np.stack( pair['Q_code'].values ).astype(np.float32)

decoder_input_data = np.stack( pair['A_code'].values )[:,:-1,:].astype(np.float32)
decoder_target_data = np.stack( pair['A_code'].values )[:,1:,:].astype(np.float32)

encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape

((1509, 27, 128), (1509, 26, 128), (1509, 26, 128))

## Строим нейросеть

In [26]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [27]:
latent_dim = 256  # размер сети

In [28]:
encoder_inputs = Input(shape=(None, w2v_size))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [29]:
from keras import backend as K
def custom_activation(x):  return (K.tanh(x) * 4)

# model.add(Dense(32 , activation=custom_activation))

Using TensorFlow backend.


In [30]:
decoder_inputs = Input(shape=(None, w2v_size))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state=encoder_states)

# decoder_dense = Dense(w2v_size)
# decoder_dense = Dense(w2v_size, activation='softmax')
# decoder_dense = Dense(w2v_size, activation='tanh')
# decoder_dense = Dense(w2v_size, activation='sigmoid')
decoder_dense = Dense(w2v_size, activation=custom_activation)

decoder_outputs = decoder_dense(decoder_outputs)

In [31]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 394240      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  394240      input_2[0][0]                    
                                                                 lstm[0][1]                       
          

In [33]:
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.compile(loss='mse', optimizer='rmsprop')
model.compile(loss='mse', optimizer='adam')
# model.compile(loss='mse', optimizer='sgd')


Instructions for updating:
Use tf.cast instead.


In [34]:
%%time 

history = model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=100,
          epochs=100,
          validation_split=0.1
        )

Train on 1358 samples, validate on 151 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
1358/1358 [==============================] - 5s 4ms/sample - loss: 0.1773 - val_loss: 0.0334
Epoch 2/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0245 - val_loss: 0.0197
Epoch 3/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0165 - val_loss: 0.0158
Epoch 4/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0140 - val_loss: 0.0140
Epoch 5/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0127 - val_loss: 0.0130
Epoch 6/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0118 - val_loss: 0.0122
Epoch 7/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0112 - val_loss: 0.0116
Epoch 8/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0107 - val_loss: 0.0112
Epoch 9/100
1358/1358 [==============================] - 4s 3ms/s

1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0077
Epoch 76/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0076
Epoch 77/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0076
Epoch 78/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0076
Epoch 79/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0077
Epoch 80/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0076
Epoch 81/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0072 - val_loss: 0.0076
Epoch 82/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0072 - val_loss: 0.0077
Epoch 83/100
1358/1358 [==============================] - 4s 3ms/sample - loss: 0.0071 - val_loss: 0.0076
Epoch 84/100
1358/1358 [==============================] - 4

## Проверяем результат

In [35]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model( [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [36]:
def decode_sequence(input_seq):
    # генерируем состояние энкодера
    states_value = encoder_model.predict(input_seq)

    # вход декодера - последовательность из одного слова GO
    output_w2v = w2v_a.wv['<GO>'].reshape([1,1,w2v_size])

    # выходная последовательность
    decoded_sentence = []
    
    for i in range(sent_len_a_max): 
        output_w2v, h, c = decoder_model.predict([output_w2v] + states_value)

        # декодируем cлово
        cc = output_w2v.reshape(w2v_size)
        # cc = cc*a_fact
        w = w2v_a.wv.similar_by_vector(cc)[0][0] 
        
        # если очередное слово это EOS
        if(w=='<EOS>'): break # то завершаем работу

        decoded_sentence.append(w)

        # обновляем состояние сети
        states_value = [h, c]

    return ' '.join(decoded_sentence)

In [37]:
ii = np.random.permutation(len(encoder_input_data))[:10]
for seq_index in ii:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print( pair.iloc[seq_index]['Q'],' -> ', decoded_sentence )

/usr/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


И молоко купил?  ->  с <GO> нет ?
Но она может и не прекратиться.  ->  с <GO> нет ?
Это настолько известно, что даже скучно.  ->  с <GO> нет ?
Приберем лучше, а то вон какой тут свинарник.  ->  с <GO> нет ?
И в то же время естественно. Мы же не знали друг друга.  ->  с <GO> нет ? . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Разговор будет долгий?  ->  с <GO> нет ?
Ну? Вы же хозяин. Может, пригласите меня сесть за стол и откроете бутылку?  ->  с <GO> нет ? . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
ты опять можешь разориться. Так что лучше разделить имущество сейчас.  ->  с <GO> нет ? . <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Я скоро вернусь. Жди.  ->  с <GO> нет ?
Ну, я пошел.  ->  с <GO> нет ?


----

In [38]:
# import matplotlib.pyplot as plt

In [39]:
# history_dict = history.history
# history_dict.keys()

In [40]:
# # acc = history.history['acc']
# #val_acc = history.history['val_acc']
# loss = history.history['loss']
# val_loss = history.history['val_loss']

# epochs = range(1, len(loss) + 1)
# plt.plot(epochs, loss, 'b', label='Training loss')
# plt.plot(epochs, val_loss, 'r', label='Validation loss')
# plt.title('Training and validation loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.show()

In [41]:
# plt.clf()   # clear figure
# acc_values = history_dict['acc']
# val_acc_values = history_dict['val_acc']

# plt.plot(epochs, acc, 'b', label='Training acc')
# plt.plot(epochs, val_acc, 'r', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

---

In [42]:
# q_min = encoder_input_data.min()
# q_max = encoder_input_data.max()
# encoder_input_data = (encoder_input_data-q_min)/(q_max-q_min)

# q_fact = np.max( [np.abs(encoder_input_data.max()), np.abs(encoder_input_data.min())] )
# encoder_input_data = encoder_input_data/q_fact

In [43]:
# encoder_input_data.min(),encoder_input_data.max()

In [44]:
# a_min = decoder_input_data.min()
# a_max = decoder_input_data.max()
# decoder_input_data  = (decoder_input_data-a_min)/(a_max-a_min)
# decoder_target_data = (decoder_target_data-a_min)/(a_max-a_min)

# a_fact = np.max( [np.abs(decoder_input_data.max()), np.abs(decoder_input_data.min())] )
# decoder_input_data  = decoder_input_data/a_fact
# decoder_target_data = decoder_target_data/a_fact

In [45]:
# decoder_target_data.min(),decoder_target_data.max()